In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import matplotlib.pyplot as plt
import csv
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display
import re
from tqdm.notebook import tqdm
import networkx as nx
import numpy as np
from datetime import datetime, timedelta
import os

# functions

In [22]:
def save_image(url, path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(path, 'wb') as file:
            for chunk in response:
                file.write(chunk)
    # else:
        # print(f"Could not download image from {url}")
        

# main

In [2]:
stamps = pd.read_csv('./csv_files/all_stamps_info_concat.csv', index_col=0)

C:\Users\Robin\AppData\Local\Temp\ipykernel_176\1845264919.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  stamps = pd.read_csv('./csv_files/all_stamps_info_concat.csv', index_col=0)


In [20]:
# get unique watermarks of "Aaland"
watermarks_ = stamps[stamps['Country'] == 'Aaland']
watermarks_ = watermarks_[watermarks_['Watermark'].notnull()]
watermarks_

,ID,SetDescription,Date,Year,Watermark,Sheetsize,Design,Engraving,Perforation,Type,...,StampsIssued,Mint,Unused,Used,LetterFDC,Currency,GroupID,Country,Category,YearSpan
No,,,,,,,,,,,,,,,,,,,,,
5,a_s_0005,Franking Label,4. January,1993,1.0,NaN,NaN,NaN,Imperforated,E,...,NaN,1.25,-,1.50,-,EUR,256042,Aaland,Franking labels,1993
5A,a_s_0005A,Franking Label,4. January,1993,1.0,NaN,NaN,NaN,Imperforated,E1,...,NaN,1.25,-,1.50,-,EUR,256042,Aaland,Franking labels,1993


In [3]:
stamps.head()

,ID,SetDescription,Date,Year,Watermark,Sheetsize,Design,Engraving,Perforation,Type,...,StampsIssued,Mint,Unused,Used,LetterFDC,Currency,GroupID,Country,Category,YearSpan
No,,,,,,,,,,,,,,,,,,,,,
1,a_s_0001,Franking Label,29. October,1984,NaN,NaN,NaN,NaN,Imperforated,A,...,NaN,1.50,-,1.75,-,EUR,256031,Aaland,Franking labels,1984
2,a_s_0002,Franking Label,4. January,1988,NaN,NaN,NaN,NaN,Imperforated,B,...,NaN,2.00,-,2.25,-,EUR,256035,Aaland,Franking labels,1988
3,a_s_0003,Franking Label,2. January,1989,NaN,NaN,NaN,NaN,Imperforated,C,...,NaN,1.50,-,1.75,-,EUR,256037,Aaland,Franking labels,1989
3A,a_s_0003A,Franking Label,2. January,1989,NaN,NaN,NaN,NaN,Imperforated,C1,...,NaN,18.00,-,10.00,-,EUR,256037,Aaland,Franking labels,1989
4,a_s_0004,Franking Label,1. March,1990,NaN,NaN,NaN,NaN,Imperforated,D,...,NaN,1.25,-,1.50,-,EUR,256040,Aaland,Franking labels,1990


In [10]:
# get all unique values in the 'Watermark' (except NaN) for each country (column 'Country')
watermarks = stamps.groupby('Country')['Watermark'].unique().apply(lambda x: [i for i in x if i == i]).reset_index()
# convert to integers
watermarks['Watermark'] = watermarks['Watermark'].apply(lambda x: [int(i) for i in x])

In [21]:
watermarks.head()

,Country,Watermark
0,AMG Naples,[1]
1,AMG Sicily,[]
2,Aaland,[1]
3,Aberdeen,[]
4,Abingdon,[]


In [23]:
# get all watermark images from "https://www.stampworld.com/media/catalogue/[Country with - instead of space]/Postage-stamps/wm[watermark].gif" and save them if it worked in ./images/[country with spaces]/watermarks/[watermark].gif
for index, row in tqdm(watermarks.iterrows(), total=watermarks.shape[0], desc="Processing"):
    country = row['Country']
    watermarks_list = row['Watermark']
    watermarks_path = os.path.join("./watermarks_all", country)
    os.makedirs(watermarks_path, exist_ok=True)
    for watermark in watermarks_list:
        watermark_path = os.path.join(watermarks_path, f"{watermark}.gif")
        # if watermark is just one digit, add a 0 in front of it
        if watermark < 10:
            watermark = f"0{watermark}"
        watermark_url = f"https://www.stampworld.com/media/catalogue/{country.replace(' ', '-')}/Postage-stamps/wm{watermark}.gif"
        save_image(watermark_url, watermark_path)

Processing:   0%|          | 0/1139 [00:00<?, ?it/s]